In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def strzero(num):
    num = str(num)
    if len(num) == 1:
        num = '0'+num
    return(num)

In [7]:
year = 2018
month = 9
day = 12
date = '20180912'
schedURL = 'https://www.espn.com/wnba/scoreboard/_/date/'+str(year)+strzero(month)+strzero(day)

In [8]:
#df = []
#date = ['20200806']

#    for date in dates:
#schedURL = f"http://www.espn.com/wnba/fixtures/_/date/{date}"
page = requests.get(schedURL)
soup = BeautifulSoup(page.content, 'html.parser')

In [9]:
soup


<!DOCTYPE html>

<html class="no-icon-fonts" lang="en">
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="IE=edge,chrome=1" http-equiv="x-ua-compatible"/>
<meta content="initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="origin-when-cross-origin" name="referrer"/>
<link href="https://www.espn.com/wnba/scoreboard" rel="canonical"/>
<title>WNBA Women's Basketball Scores - WNBA Scoreboard - ESPN</title>
<meta content="Real-time WNBA Women's Basketball scores on ESPN" name="description">
<meta content="WNBA scores, WNBA scoreboard, Women's Basketball scores, Women's Basketball scoreboard" name="news_keywords">
<meta content="WNBA scores, WNBA scoreboard, Women's Basketball scores, Women's Basketball scoreboard" name="keywords"/>
<meta content="116656161708917" property="fb:app_id"/>
<meta content="ESPN.com" property="og:site_name"/>
<meta content="https://www.espn.com/wnba/scoreboard" property="og:url"/>
<meta 

In [ ]:
#scrape url
yeardict = {'2019':'401104'}
GameID = '401230763'
URL = 'https://www.espn.com/wnba/playbyplay?gameId='+GameID
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
#get team abbreviations from top
teams = [x.text for x in soup.find_all('span', class_='abbrev')]
teams = {teams[0]:'away',teams[1]:'home'}

In [ ]:
#scrape the shot description table
times = [x.text for x in soup.find_all('td', class_='time-stamp')]
plays = [x.text for x in soup.find_all('td', class_='game-details')]
score = [x.text for x in soup.find_all('td', class_='combined-score')]

#get team names from the logo image urls
logourl = [x.find('img').get('src') for x in soup.find_all('td', class_='logo')]
team = [re.match(r"https.*/wnba/500/(?P<team>[a-z]{3})",x).group('team').upper() for x in logourl]
homeoraway = [teams[x] for x in team]

#convert to df
shotdescriptions = pd.DataFrame({'Time': times,
                      'EventDescription': plays,
                      'Score': score,
                      'EventTeam': team,
                      'HomeOrAway':homeoraway})
shotdescriptions['GameID'] = GameID

In [ ]:
#get shot chart data for away team
awayshots = soup.find('ul', class_='shots away-team')
awayshots = awayshots.find_all('li')
awayshotchart = []
for row in awayshots:
    coordtext = row.get('style')
    match = re.match(r".*left:(?P<xcoord>[0-9\.]+)%;top:(?P<ycoord>[0-9\.]+)%;",coordtext)
    xcoord = match.group('xcoord')
    ycoord = match.group('ycoord')
    ycoord = str(100-float(ycoord))
    row_data = [row.get('class')[0],
                row.get('data-homeaway'),
                row.get('data-period'),
                row.get('data-shooter'),
                row.get('data-text'),
                row.get('id'),
                xcoord,
                ycoord]
    awayshotchart.append(row_data)

#get shot chart data for home team
homeshots = soup.find('ul', class_='shots home-team')
homeshots = homeshots.find_all('li')
homeshotchart = []
for row in homeshots:
    coordtext = row.get('style')
    match = re.match(r".*left:(?P<xcoord>[0-9\.]+)%;top:(?P<ycoord>[0-9\.]+)%;",coordtext)
    xcoord = match.group('xcoord')
    ycoord = match.group('ycoord')
    ycoord = str(100-float(ycoord))
    row_data = [row.get('class')[0],
                row.get('data-homeaway'),
                row.get('data-period'),
                row.get('data-shooter'),
                row.get('data-text'),
                row.get('id'),
                xcoord,
                ycoord]
    homeshotchart.append(row_data)

awayshotchart = pd.DataFrame(awayshotchart)
homeshotchart = pd.DataFrame(homeshotchart)

In [ ]:
#name columns for shot chart data
awayshotchart.columns = ['ShotOutcome','EventTeam','Quarter','pid','EventDescription','ShotID','xcoord','ycoord']
homeshotchart.columns = ['ShotOutcome','EventTeam','Quarter','pid','EventDescription','ShotID','xcoord','ycoord']

In [ ]:
#join to shot descriptions
playindex = 0
for row in range(len(awayshotchart)):
    not_found = True
    while not_found:
        if shotdescriptions.loc[playindex,'EventDescription']==awayshotchart.loc[row,'EventDescription']:
            if shotdescriptions.loc[playindex,'HomeOrAway']=='away':
                shotdescriptions.loc[playindex,'ShotOutcome']=awayshotchart.loc[row,'ShotOutcome']
                shotdescriptions.loc[playindex,'Quarter']=awayshotchart.loc[row,'Quarter']
                shotdescriptions.loc[playindex,'pid']=awayshotchart.loc[row,'pid']
                shotdescriptions.loc[playindex,'ShotID']=awayshotchart.loc[row,'ShotID']
                shotdescriptions.loc[playindex,'xcoord']=awayshotchart.loc[row,'xcoord']
                shotdescriptions.loc[playindex,'ycoord']=awayshotchart.loc[row,'ycoord']
                not_found = False
        playindex += 1       

playindex = 0
for row in range(len(homeshotchart)):
    not_found = True
    while not_found:
        if shotdescriptions.loc[playindex,'EventDescription']==homeshotchart.loc[row,'EventDescription']:
            if shotdescriptions.loc[playindex,'HomeOrAway']=='home':
                shotdescriptions.loc[playindex,'ShotOutcome']=homeshotchart.loc[row,'ShotOutcome']
                shotdescriptions.loc[playindex,'Quarter']=homeshotchart.loc[row,'Quarter']
                shotdescriptions.loc[playindex,'pid']=homeshotchart.loc[row,'pid']
                shotdescriptions.loc[playindex,'ShotID']=homeshotchart.loc[row,'ShotID']
                shotdescriptions.loc[playindex,'xcoord']=homeshotchart.loc[row,'xcoord']
                shotdescriptions.loc[playindex,'ycoord']=homeshotchart.loc[row,'ycoord']
                not_found = False
        playindex += 1              

In [ ]:
quarter = 1
shotdescriptions.loc[0,'AwayScore'] = 0
shotdescriptions.loc[0,'HomeScore'] = 0
for row in range(1,len(shotdescriptions)):
    if shotdescriptions.loc[row,'Time'] > shotdescriptions.loc[row-1,'Time']:
        quarter += 1
    shotdescriptions.loc[row,'Quarter'] = quarter
    match = re.match(("([0-9]+) - ([0-9]+)"),shotdescriptions.loc[row,'Score'])
    shotdescriptions.loc[row,'AwayScore'] = match.group(1)
    shotdescriptions.loc[row,'HomeScore'] = match.group(2)

In [ ]:
shotdescriptions